# Expérience de vérification automatisée de la qualité lors de la collecte de vidéos YouTube

Ce notebook implémente une chaîne automatisée de collecte et de contrôle qualité des vidéos issues de YouTube, dans le cadre de la création d'archives de jouabilité.

L'objectif est de garantir l'intégrité, la lisibilité et la cohérence des vidéos collectées tout en réduisant le travail manuel.  
Pour chaque vidéo, le notebook effectue les étapes suivantes :

1. **Téléchargement automatisé** via `yt-dlp` en format MP4.
2. **Contrôle d'intégrité** en calculant une empreinte SHA-256.
3. **Vérification technique** de la lisibilité avec `ffmpeg`.
4. **Collecte de métadonnées** (taille, durée, statut) dans un fichier CSV.

Le notebook est conçu pour être exécuté sur Google Colab et peut être adapté à tout corpus de vidéos YouTube destiné à des études de jouabilité ou à la constitution d'archives numériques.

#1. Installation et importation des dépendances

In [ ]:
# Installer les packages nécessaires
!pip install yt-dlp
!apt-get install ffmpeg -y

import os
import hashlib
import subprocess
import pandas as pd
from datetime import datetime


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.9/175.9 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 45.0 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


#2. Configuration des chemins et paramètres

In [ ]:
# Dossier de travail
output_dir = "/content/archives_jeu"
os.makedirs(output_dir, exist_ok=True)

# URL YouTube exemple from the New World Aeternum official YouTube Channel
youtube_urls = [
    "https://www.youtube.com/watch?v=VDOjE9mTY5Y", # 19 min 42 : Forged in Aeternum - Community Q&A (July 2025)
    "https://www.youtube.com/watch?v=RvzSZupZcDI", # 21 min 04 : New World: Dev Update - July 2025
    "https://www.youtube.com/watch?v=2qRVEk_IiJU" # 17 min 41 : New World: Aeternum - Capture the Flag
]

# Fichier CSV pour métadonnées
metadata_file = os.path.join(output_dir, "metadata.csv")

# Initialiser un DataFrame pour stocker les métadonnées
metadata_df = pd.DataFrame(columns=[
    "url", "filename", "sha256", "size_bytes", "duration_seconds", "status", "timestamp"
])


#3. Fonctions utilitaires

In [ ]:
# Fonction pour calculer l'empreinte SHA-256
def compute_sha256(file_path):
    sha256_hash = hashlib.sha256()
    with open(file_path, "rb") as f:
        for byte_block in iter(lambda: f.read(4096), b""):
            sha256_hash.update(byte_block)
    return sha256_hash.hexdigest()

# Fonction pour obtenir la durée de la vidéo via ffmpeg
def get_video_duration(file_path):
    result = subprocess.run(
        ["ffprobe", "-v", "error", "-select_streams", "v:0",
         "-show_entries", "format=duration", "-of", "default=noprint_wrappers=1:nokey=1", file_path],
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT
    )
    try:
        return float(result.stdout)
    except:
        return None


#4. Téléchargement et contrôle automatisé

In [ ]:
import yt_dlp

for url in youtube_urls:
    timestamp = datetime.now().isoformat()
    try:
        ydl_opts = {
            'format': 'mp4',
            'outtmpl': os.path.join(output_dir, '%(title)s.%(ext)s'),
            'quiet': True
        }
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            info_dict = ydl.extract_info(url, download=True)
            filename = ydl.prepare_filename(info_dict)

        sha256 = compute_sha256(filename)
        size_bytes = os.path.getsize(filename)
        duration = get_video_duration(filename)
        status = "OK" if duration else "Corrupted"

    except Exception as e:
        filename = None
        sha256 = None
        size_bytes = None
        duration = None
        status = f"Failed: {e}"

    metadata_df = pd.concat([metadata_df, pd.DataFrame([{
        "url": url,
        "filename": filename,
        "sha256": sha256,
        "size_bytes": size_bytes,
        "duration_seconds": duration,
        "status": status,
        "timestamp": timestamp
    }])], ignore_index=True)

# Sauvegarder les métadonnées
metadata_df.to_csv(metadata_file, index=False)
metadata_df.head()


/tmp/ipython-input-3544889091.py:27: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



,url,filename,sha256,size_bytes,duration_seconds,status,timestamp
0,https://www.youtube.com/watch?v=VDOjE9mTY5Y,/content/archives_jeu/Forged in Aeternum - Co...,2698a93ef75c670ce08d0ab9954bd3ae5387c3d3f7b75e...,53777879,1182.035,OK,2025-08-21T21:43:39.823619
1,https://www.youtube.com/watch?v=RvzSZupZcDI,/content/archives_jeu/New World： Dev Update -...,9afa59e7aa3b751af68777403553ef3e85914ac75b300d...,64221820,1264.861,OK,2025-08-21T21:43:54.526229
2,https://www.youtube.com/watch?v=2qRVEk_IiJU,/content/archives_jeu/New World： Aeternum - Ca...,fd141a285a064c163b0a43f494d33e1eabcaf479e00400...,82445097,1061.686,OK,2025-08-21T21:44:13.557902


#5. Rapports et contrôle statistique simple

In [ ]:
# Nombre de vidéos téléchargées
print(f"Total vidéos : {len(metadata_df)}")

# Statut
print(metadata_df['status'].value_counts())

# Durée totale
print(f"Durée totale (s) : {metadata_df['duration_seconds'].sum()}")


Total vidéos : 3
status
OK    3
Name: count, dtype: int64
Durée totale (s) : 3508.5820000000003


#6. Visualisation plus complexe des statistiques de contrôle

In [ ]:
!pip install plotly

import plotly.express as px

# Graphique 1 : Statut des vidéos
fig_status = px.histogram(metadata_df, x='status', title='Statut des vidéos téléchargées')
fig_status.show()

# Graphique 2 : Distribution des durées
fig_duration = px.histogram(metadata_df, x='duration_seconds', nbins=20,
                            title='Distribution des durées des vidéos (s)')
fig_duration.show()

# Graphique 3 : Taille des fichiers
fig_size = px.histogram(metadata_df, x='size_bytes', nbins=20,
                        title='Distribution des tailles des vidéos (octets)')
fig_size.show()


##6.1. Diagramme de type "status par durée et taille"
Chaque point = une vidéo
Axe X = durée, Axe Y = taille du fichier
Couleur = statut (OK / Corrompu / Failed)
Taille du point = volume du fichier (optionnel)
Permet de repérer d’un coup les vidéos anormales.

In [ ]:
fig_scatter = px.scatter(
    metadata_df,
    x='duration_seconds',
    y='size_bytes',
    color='status',
    hover_data=['filename', 'url'],
    size=metadata_df['size_bytes'].astype(float), # Explicitly convert to float
    title='Durée vs Taille des vidéos (couleur = statut)'
)
fig_scatter.show()

##6.2. Chronologie des téléchargements
Affiche quand chaque vidéo a été collectée
Permet de suivre l’activité et de détecter des interruptions ou échecs dans le temps.

In [ ]:
metadata_df['timestamp'] = pd.to_datetime(metadata_df['timestamp'])
fig_timeline = px.scatter(
    metadata_df,
    x='timestamp',
    y='filename',
    color='status',
    title='Chronologie des téléchargements et statut'
)
fig_timeline.show()
